In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv("poke_battle_logger_teams.csv")
df2 = pd.read_csv("battle_database_results.csv")

In [ ]:
df

,your_team
0,"ヒスイドレディア,ハバタクカミ,パオジアン,ヒスイウォーグル,ドラパルト,ヒードラン"
1,"ガチグマ赫月,ギャラドス,イイネイヌ,クエスパトラ,炎オーガポン,カバルドン"
2,"ガチグマ赫月,ギャラドス,トドロクツキ,クエスパトラ,岩オーガポン,エルレイド"
3,"キノガッサ,ハバタクカミ,サーフゴー,ヘイラッシャ,ヌメルゴン,ミミズズ"
4,"ヒスイドレディア,ハバタクカミ,パオジアン,ウォッシュロトム,ドラパルト,ヒードラン"
...,...
650,"ジバコイル,セグレイブ,ハバタクカミ,テツノツツミ,サーフゴー,マスカーニャ"
651,"ハッサム,ハバタクカミ,テツノツツミ,カイリュー,イーユイ,イダイナキバ"
652,"パオジアン,キョジオーン,サケブシッポ,サザンドラ,カイリュー,ミミッキュ"
653,"パオジアン,トドロクツキ,イダイナキバ,カイリュー,ミミッキュ,イーユイ"


In [ ]:
# ポケモンの並び
item_sequences = [sorted(v.split(",")) for v in df["your_team"].values.tolist()] + df2.values.tolist()
item_sequences = pd.Series(item_sequences).drop_duplicates().values.tolist()

In [ ]:
import random

random_add_num = 6

shuffled_sequences = []
for sequence in item_sequences:
    for _ in range(random_add_num):
        shuffled = sequence.copy()
        random.shuffle(shuffled)
        shuffled_sequences.append(shuffled)

In [ ]:
augmented_item_sequences = item_sequences + shuffled_sequences

In [ ]:
from transformers import BertTokenizer, BertConfig, BertForMaskedLM
import os

# すべてのユニークなアイテムIDを取得
unique_items = set(sum([seq for seq in augmented_item_sequences], []))

# 特別トークンを含む語彙リストを作成
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + list(unique_items)

os.makedirs("custom_vocab", exist_ok=True)
with open("custom_vocab/vocab.txt", "w") as vocab_file:
    for token in vocab:
        vocab_file.write(token + "\n")

# トークナイザーの初期化
tokenizer = BertTokenizer(vocab_file="custom_vocab/vocab.txt", do_lower_case=False)
tokenizer.add_tokens(vocab)

# トークナイザーの保存（必要に応じて）
os.makedirs("custom_tokenizer", exist_ok=True)
tokenizer.save_pretrained("custom_tokenizer")

# BERTモデルの初期化
config = BertConfig(vocab_size=len(tokenizer))
model = BertForMaskedLM(config)
model.resize_token_embeddings(len(tokenizer))  # トークナイザーの語彙サイズに合わせてモデルのトークン埋め込みを調整

Embedding(281, 768, padding_idx=0)

In [ ]:
tokenizer("ヒスイドレディア ハバタクカミ パオジアン ヒスイウォーグル ドラパルト ヒードラン")

{'input_ids': [2, 58, 51, 118, 132, 227, 96, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
from sklearn.model_selection import train_test_split

random.shuffle(augmented_item_sequences)
train_data,test_data = train_test_split(augmented_item_sequences, random_state=42, test_size=0.1)
train_data,eval_data = train_test_split(train_data, random_state=42, test_size=0.1)

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertConfig, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset

# トークナイズされたデータセットを作成
def tokenize_function(examples):
    return tokenizer(examples["text"])

# データセットの作成
ds = Dataset.from_dict({"text": [" ".join(seq) for seq in train_data]})
train_tokenized_ds = ds.map(tokenize_function, batched=True)

eval_ds = Dataset.from_dict({"text": [" ".join(seq) for seq in eval_data]})
eval_tokenized_ds = eval_ds.map(tokenize_function, batched=True)

# 3. MLM用のデータコレーターの定義
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# 4. BERTモデルの初期化
config = BertConfig(vocab_size=len(tokenizer))
model = BertForMaskedLM(config)
model.resize_token_embeddings(len(tokenizer))

# 5. トレーニングの設定
training_args = TrainingArguments(
    output_dir="my_awesome_pokemon_party_mlm_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=50,
    weight_decay=0.01,
    save_total_limit = 5,
    push_to_hub=False  # モデルをHugging Face Hubにアップロードしない場合はFalseに設定
)

Map:   0%|          | 0/15955 [00:00<?, ? examples/s]

Map:   0%|          | 0/1773 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=eval_tokenized_ds,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.786000,3.706618
2,3.510600,3.506225
3,3.338900,3.297328
4,3.242200,3.300004
5,3.173000,nan
6,3.051600,3.090165
7,2.992800,3.113814
8,2.950900,3.000655
9,2.898800,3.004670
10,2.810500,2.934143


TrainOutput(global_step=99750, training_loss=2.1641170681831534, metrics={'train_runtime': 8491.8236, 'train_samples_per_second': 93.943, 'train_steps_per_second': 11.747, 'total_flos': 3279648877428000.0, 'train_loss': 2.1641170681831534, 'epoch': 50.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 4.24


In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "/content/my_awesome_pokemon_party_mlm_model/checkpoint-99500", tokenizer=tokenizer)

In [ ]:
text = "グライオン ハピナス クレセリア クレベース モロバレル [MASK]"
mask_filler(text, top_k=10)

[{'score': 0.5603667497634888,
  'token': 49,
  'token_str': 'ドヒドイデ',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ドヒドイデ'},
 {'score': 0.30369824171066284,
  'token': 147,
  'token_str': 'ヘイラッシャ',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ヘイラッシャ'},
 {'score': 0.035393208265304565,
  'token': 48,
  'token_str': 'ヤドラン',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ヤドラン'},
 {'score': 0.023812824860215187,
  'token': 30,
  'token_str': 'ママンボウ',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ママンボウ'},
 {'score': 0.017130794003605843,
  'token': 209,
  'token_str': 'ラウドボーン',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ラウドボーン'},
 {'score': 0.011878104880452156,
  'token': 68,
  'token_str': 'ピクシー',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ピクシー'},
 {'score': 0.0030177668668329716,
  'token': 227,
  'token_str': 'ドラパルト',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ドラパルト'},
 {'score': 0.0026856872718781233,
  'token': 154,
  'token_str': 'ドオー',
  'sequence': 'グライオン ハピナス クレセリア クレベース モロバレル ドオー'},
 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("fufufukakaka/pokemon_party_BERT")

model.safetensors:   0%|          | 0.00/345M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

events.out.tfevents.1705103054.8ff69832cda1.695.5:   0%|          | 0.00/50.0k [00:00<?, ?B/s]

events.out.tfevents.1705111573.8ff69832cda1.695.6:   0%|          | 0.00/364 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fufufukakaka/my_awesome_pokemon_party_mlm_model/commit/6b0037781378abb814e822f48d3f6598753dd4cb', commit_message='fufufukakaka/pokemon_party_BERT', commit_description='', oid='6b0037781378abb814e822f48d3f6598753dd4cb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("fufufukakaka/pokemon_party_BERT")

CommitInfo(commit_url='https://huggingface.co/fufufukakaka/pokemon_party_BERT/commit/aa6f8e5ae691b6e0ba1ea8fbccdf339a93ae2ce0', commit_message='Upload tokenizer', commit_description='', oid='aa6f8e5ae691b6e0ba1ea8fbccdf339a93ae2ce0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import shutil

shutil.make_archive('pokemon_party_BERT', format='zip', root_dir='/content/my_awesome_pokemon_party_mlm_model/checkpoint-99500')

'/content/pokemon_party_BERT.zip'

In [ ]:
dataset = Dataset.from_dict({"text": [" ".join(seq) for seq in augmented_item_sequences]})
dataset.push_to_hub("fufufukakaka/pokemon_party_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/fufufukakaka/pokemon_party_dataset/commit/9f0b30e706a52b3f9b40474700cadcf22014846c', commit_message='Upload dataset', commit_description='', oid='9f0b30e706a52b3f9b40474700cadcf22014846c', pr_url=None, pr_revision=None, pr_num=None)